<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/Problema_de_corte_de_estoque.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema de corte de estoque

Neste notebook, implementamos um modelo de programação linear inteira para o clássico problema de corte de estoque unidimensional, dado a seguir:

\begin{align}
\min \quad z & = x_1+x_2+x_3+x_3+x_4+x_5 \\
\text{s.a.} &\\
& x_1+2x_3+x_4 \geq 10\\
& x_1+2x_3+x_4 \geq 12\\
& x_1+x_2 \geq 15\\
& 2x_2+x_4+2x_5 \geq 31\\
& x_3+x_5 \geq 17 \\
& x_1, x_2, x_3, x_4, x_5 \in \{0,1,2, \dots\}
\end{align}

## Instalação das bibliotecas necessárias

In [1]:
!pip install pyomo
!apt-get install coinor-cbc

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 14.0 MB/s 
     |████████████████████████████████| 49 kB 4.5 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  coinor-libcbc3 coinor-libcgl1 coinor-libclp1 coinor-libcoinutils3v5
  coinor-libosi1v5
The following NEW packages will be installed:
  coinor-cbc coinor-libcbc3 coinor-libcgl1 coinor-libclp1
  coinor-libcoinutils3v5 coinor-libosi1v5
0 upgraded, 6 newly installed, 0 to remove and 42 not upgraded.
Need to get 2,737 kB of archives.
After this operation, 8,130 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 coinor-libcoinutils3v5 amd64 2.10.14+re

In [2]:
import pyomo.environ as pyEnv
import numpy as np

## Declaração dos dados do problema

Note que cada padrão é uma coluna da matriz ```padroes```:



In [3]:
padroes = np.array([[1, 0, 0, 3, 1],
                    [1, 0, 2, 1, 0],
                    [1, 1, 0, 0, 0],
                    [0, 2, 0, 1, 2],
                    [0, 0, 1, 0, 1]])

demandas = np.array([10, 12, 15, 31, 17])

## Declaração do modelo

In [4]:
m = padroes.shape[0]   ##Número de itens diferentes
n = padroes.shape[1]   ##Número de padroes é o número de colunas

##Criação da instância do modelo
modelo = pyEnv.ConcreteModel()

##Criação dos índices
modelo.Indices = range(n)
modelo.Indices2 = range(m)

##Variáveis de decisão
modelo.x = pyEnv.Var(modelo.Indices, within = pyEnv.NonNegativeIntegers)    ##Note a declaração das variáveis como inteiras não negativas

##Função-objetivo
modelo.Objetivo = pyEnv.Objective(expr = sum(modelo.x[i] for i in modelo.Indices),
                                  sense = pyEnv.minimize)    ##Note a direção de minimização

##Note aqui a declaração das restrições. Para não declarar cada restrição individualmente
##definimos uma função que gera as restrições.

def restricao(modelo, i):
    return sum(padroes[i][j]*modelo.x[j] for j in modelo.Indices) >= demandas[i]

##Note que para cada valor em modelo.Indices2 será criada uma restrição,
##em que o índice i na função restricao assumirá um valor em modelo.Indices2
modelo.rest = pyEnv.Constraint(modelo.Indices2, rule = restricao)


##Solução do modelo

In [5]:
solver = pyEnv.SolverFactory('cbc')
result_objetivo = solver.solve(modelo, tee=True)

Welcome to the CBC MILP Solver 
Version: 2.9.9 
Build Date: Aug 21 2017 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmpqnmclu1u.pyomo.lp -stat=1 -solve -solu /tmp/tmpqnmclu1u.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 5 (-1) rows, 5 (-1) columns and 13 (-1) elements
Statistics for presolved model
Original problem has 5 integers (0 of which binary)
Presolved problem has 5 integers (0 of which binary)
==== 0 zero objective 1 different
5 variables have objective of 1
==== absolute objective values 1 different
5 variables have objective of 1
==== for integers 0 zero objective 1 different
5 variables have objective of 1
==== for integers absolute objective values 1 different
5 variables have objective of 1
===== end objective counts


Problem has 5 rows, 5 columns (5 with objective) and 13 elements
Column breakdown:
5 of type 0.0->inf, 0 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fi

## Print do resultado

In [7]:
lista = list(modelo.x.keys())
print('\nVariaveis: ')
for i in lista:
    print('x_'+str(i+1), '---', modelo.x[i]())

print('\nValor da função objetivo =', modelo.Objetivo())



Variaveis: 
x_1 --- 0.0
x_2 --- 15.0
x_3 --- 7.0
x_4 --- 0.0
x_5 --- 10.0

Valor da função objetivo = 32.0
